In [4]:
import os
import sys
from functools import singledispatch
import warnings

import pandas as pd
import numpy as np
import math
import pyproj

sys.path.insert(0, "/home/tthatcher/Desktop/Projects/?Plio/plio")

from plio.examples import get_path
from plio.io.io_bae import read_gpf, read_ipf

In [5]:
# Reads a .atf file and outputs all of the 
# .ipf, .gpf, .sup, .prj, and path to locate the 
# .apf file (should be the same as all others) 
def read_atf(atf_file):
    with open(atf_file) as f:

        files = []
        ipf = []
        sup = []
        files_dict = []
        
        # Grabs every PRJ, GPF, SUP, and IPF image from the ATF file
        for line in f:
            if line[-4:-1] == 'prj' or line[-4:-1] == 'gpf' or line[-4:-1] == 'sup' or line[-4:-1] == 'ipf' or line[-4:-1] == 'atf':
                files.append(line)
        
        files = np.array(files)
        
        # Creates appropriate arrays for certain files in the right format
        for file in files:
            file = file.strip()
            file = file.split(' ')

            # Grabs all the IPF files
            if file[1].endswith('.ipf'):
                ipf.append(file[1])

            # Grabs all the SUP files
            if file[1].endswith('.sup'):
                sup.append(file[1])

            files_dict.append(file)

        # Creates a dict out of file lists for GPF, PRJ, IPF, and ATF
        files_dict = (dict(files_dict))
        
        # Sets the value of IMAGE_IPF to all IPF images
        files_dict['IMAGE_IPF'] = ipf
        
        # Sets the value of IMAGE_SUP to all SUP images
        files_dict['IMAGE_SUP'] = sup
        
        # Sets the value of PATH to the path of the ATF file
        files_dict['PATH'] = os.path.dirname(os.path.abspath(atf_file))
        
        return files_dict

In [6]:
# converts columns l. and s. to isis
def line_sample_size(record, path):
    with open(os.path.join(path, record['ipf_file'] + '.sup')) as f:
        for i, line in enumerate(f):
            if i == 3:
                line_size = line.split(' ')
                line_size = line_size[-1].strip()
                assert int(line_size) > 0, "Line number {} from {} is a negative number: Invalid Data".format(line_size, record['ipf_file'])
                
            if i == 4:
                sample_size = line.split(' ')
                sample_size = sample_size[-1].strip()
                assert int(sample_size) > 0, "Sample number {} from {} is a negative number: Invalid Data".format(sample_size, record['ipf_file'])
                break
                
        
        line_size = int(line_size)/2.0 + record['l.'] + 1
        sample_size = int(sample_size)/2.0 + record['s.'] + 1
        return sample_size, line_size
    
# converts known to ISIS keywords
def known(record):
    if record['known'] == 0:
        return 'Free'
    
    elif record['known'] == 1 or record['known'] == 2 or record['known'] == 3:
        return 'Constrained'
    
# converts +/- 180 system to 0 - 360 system
def to_360(num):
    return num % 360

# ocentric to ographic latitudes
def oc2og(dlat, dMajorRadius, dMinorRadius):
    try:    
        dlat = math.radians(dlat)
        dlat = math.atan(((dMajorRadius / dMinorRadius)**2) * (math.tan(dlat)))
        dlat = math.degrees(dlat)
    except:
        print ("Error in oc2og conversion")
    return dlat

# ographic to ocentric latitudes
def og2oc(dlat, dMajorRadius, dMinorRadius):
    try:
        dlat = math.radians(dlat)
        dlat = math.atan((math.tan(dlat) / ((dMajorRadius / dMinorRadius)**2)))
        dlat = math.degrees(dlat)
    except:
        print ("Error in og2oc conversion")
    return dlat

# gets eRadius and pRadius from a .prj file
def get_axis(file):
    with open(file) as f:
        from collections import defaultdict

        files = defaultdict(list)
        
        for line in f:
            
            ext = line.strip().split(' ')
            files[ext[0]].append(ext[-1])
            
        eRadius = float(files['A_EARTH'][0])
        pRadius = eRadius * (1 - float(files['E_EARTH'][0]))
        
        return eRadius, pRadius
    
# function to convert lat_Y_North to ISIS_lat
def lat_ISIS_coord(record, semi_major, semi_minor):
        ocentric_coord = og2oc(record['lat_Y_North'], semi_major, semi_minor)
        coord_360 = to_360(ocentric_coord)
        return coord_360

# function to convert long_X_East to ISIS_lon
def lon_ISIS_coord(record, semi_major, semi_minor):
        ocentric_coord = og2oc(record['long_X_East'], semi_major, semi_minor)
        coord_360 = to_360(ocentric_coord)
        return coord_360

def body_fix(record, semi_major, semi_minor):
    ecef = pyproj.Proj(proj='geocent', a=semi_major, b=semi_minor)
    lla = pyproj.Proj(proj='latlon', a=semi_major, b=semi_minor)
    lon, lat, height = pyproj.transform(lla, ecef, record['long_X_East'], record['lat_Y_North'], record['ht'])
    return lon, lat, height

def apply_transformation(atf_dict, df):
    prj_file = os.path.join(atf_dict['PATH'], atf_dict['PROJECT'].split('\\')[-1])
    
    eRadius, pRadius = get_axis(prj_file)
    df['s.'], df['l.'] = (zip(*df.apply(line_sample_size, path = atf_dict['PATH'], axis=1)))
    df['known'] = df.apply(known, axis=1)
    df['lat_Y_North'] = df.apply(lat_ISIS_coord, semi_major = eRadius, semi_minor = pRadius, axis=1)
    df['long_X_East'] = df.apply(lon_ISIS_coord, semi_major = eRadius, semi_minor = pRadius, axis=1)
    df['long_X_East'], df['lat_Y_North'], df['ht'] = zip(*df.apply(body_fix, semi_major = eRadius, semi_minor = pRadius, axis = 1))

In [8]:
def socet2isis(atf_file, cube_path):
    atf_dict = read_atf(get_path(atf_file))

    gpf_file = os.path.join(atf_dict['PATH'], atf_dict['GP_FILE']);
    ipf_list = [os.path.join(atf_dict['PATH'], i) for i in atf_dict['IMAGE_IPF']]

    gpf_df = read_gpf(gpf_file)
    ipf_df = read_ipf(ipf_list)

    point_diff = ipf_df.index.difference(gpf_df.index)

    if len(point_diff) != 0:
        warnings.warn("The following points found in ipf files missing from gpf file: \n\n{}. \
                      \n\nContinuing, but these points will be missing from the control network".format(list(point_diff)))

    socet_df = ipf_df.merge(gpf_df, left_on = 'pt_id', right_on = 'point_id')

    apply_transformation(atf_dict, socet_df)
    
    return socet_df
    
socet2isis('CTX_Athabasca_Middle_step0.atf', 'Banana')

,pt_id,val,fid_val,no_obs,l.,s.,sig_l,sig_s,res_l,res_s,...,known,lat_Y_North,long_X_East,ht,sig0,sig1,sig2,res0,res1,res2
0,1_8344_8845_4r,1,0,0,636.887207,133.604004,0.000000,0.000000,0.278602,-0.244702,...,Free,128580.921397,3.391207e+06,5990.723423,0.0,0.0,4.000000,49.937532,-24.131687,639.699743
1,1_8344_8845_4r,1,0,0,38.017578,225.989258,0.000000,0.000000,-0.062556,-0.214713,...,Free,128580.921397,3.391207e+06,5990.723423,0.0,0.0,4.000000,49.937532,-24.131687,639.699743
2,1_8344_8845_4r,1,0,0,4096.101562,3221.674805,0.000000,0.000000,-0.069161,0.311984,...,Free,128580.921397,3.391207e+06,5990.723423,0.0,0.0,4.000000,49.937532,-24.131687,639.699743
3,1_8344_8845_4r,1,0,0,9564.642090,3048.304413,0.000000,0.000000,0.002209,0.194239,...,Free,128580.921397,3.391207e+06,5990.723423,0.0,0.0,4.000000,49.937532,-24.131687,639.699743
4,2_8344_8845_4r,1,0,0,733.781250,1935.229126,0.000000,0.000000,-0.087176,0.686952,...,Free,128441.872032,3.391210e+06,5983.451630,0.0,0.0,5.950495,31.803679,2.604902,626.707206
5,2_8344_8845_4r,1,0,0,127.934814,1937.150757,0.000000,0.000000,0.228660,0.105249,...,Free,128441.872032,3.391210e+06,5983.451630,0.0,0.0,5.950495,31.803679,2.604902,626.707206
6,2_8344_8845_4r,1,0,0,4193.808594,5020.347900,0.000000,0.000000,-0.012951,-0.655346,...,Free,128441.872032,3.391210e+06,5983.451630,0.0,0.0,5.950495,31.803679,2.604902,626.707206
7,2_8344_8845_4r,1,0,0,9648.685059,4511.925781,0.000000,0.000000,0.017553,-0.095367,...,Free,128441.872032,3.391210e+06,5983.451630,0.0,0.0,5.950495,31.803679,2.604902,626.707206
8,3_8344_8845_4r_mt_z,1,0,0,3675.705360,130.808105,0.000000,0.000000,-0.292195,-0.573220,...,Constrained,128549.488243,3.391178e+06,6191.069521,0.0,0.0,2.000000,21.103941,14.908475,0.160007
9,3_8344_8845_4r_mt_z,1,0,0,3077.260986,243.122803,0.000000,0.000000,-0.025129,-0.002447,...,Constrained,128549.488243,3.391178e+06,6191.069521,0.0,0.0,2.000000,21.103941,14.908475,0.160007
